In [1]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

from read_and_split_data import split_data, filter_animes_without_grade

In [2]:
def get_dataset():
    anime = pd.read_parquet("../datasets/anime.parquet")
    anime = anime[["anime_id", "type"]]
    users = pd.read_parquet("../datasets/users.parquet")
    base_df = users.merge(anime, on="anime_id", how="left")
    return base_df

base_df = get_dataset()
data = split_data(base_df)

In [3]:
kf = KFold(n_splits=5)

In [4]:
def predict_and_error(data, algo, path):
    fold = 0
    predictions_list = []
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        predictions_list.append(predictions)
        # Mean Squared Error
        accuracy.rmse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mse(predictions, verbose=True)
        # Mean Absolute Erro
        accuracy.mae(predictions, verbose=True)
        # Fraction of Concordant Pairs
        accuracy.fcp(predictions, verbose=True)
        base_df = pd.DataFrame(columns=["user", "anime", "actual", "est", "details"], data=predictions)
        base_df.to_csv(f"{path}_{fold}.csv")
        base_df.to_parquet(f"{path}_{fold}.parquet")
        fold += 1
    return predictions

In [5]:
algo_normal_predictor = SVD()

In [6]:
predictions = predict_and_error(data, algo_normal_predictor, path="../predictions/svd/anime_type_tv/no_filter/svd")

RMSE: 2.1744
MSE: 4.7278
MAE:  1.5025
FCP:  0.6817
RMSE: 2.1795
MSE: 4.7501
MAE:  1.5035
FCP:  0.6794
RMSE: 2.1814
MSE: 4.7585
MAE:  1.5042
FCP:  0.6804
RMSE: 2.1860
MSE: 4.7787
MAE:  1.5066
FCP:  0.6803
RMSE: 2.1773
MSE: 4.7404
MAE:  1.5032
FCP:  0.6814


In [7]:
base_df_without_negative = filter_animes_without_grade(base_df)

In [8]:
predictions_without_negative = predict_and_error(data, algo_normal_predictor, path="../predictions/svd/anime_type_tv/with_filter_remove_negative/svd")

RMSE: 2.1763
MSE: 4.7365
MAE:  1.5019
FCP:  0.6809
RMSE: 2.1822
MSE: 4.7620
MAE:  1.5042
FCP:  0.6810
RMSE: 2.1795
MSE: 4.7503
MAE:  1.5041
FCP:  0.6802
RMSE: 2.1817
MSE: 4.7597
MAE:  1.5046
FCP:  0.6804
RMSE: 2.1794
MSE: 4.7497
MAE:  1.5039
FCP:  0.6809
